<a href="https://colab.research.google.com/github/MoqiSheng/MoqiSheng.github.io/blob/main/qwen3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
import json

# 设置环境变量
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 设置路径
LLM_datapath = "/content/drive/MyDrive/Qwen3-8B"
prompt_data_path = "/content/data/street_prompt_72.csv"
type_save_path = "/content/drive/MyDrive/Qwen3-8B_result"

# 创建保存目录
os.makedirs(type_save_path, exist_ok=True)
checkpoint_path = os.path.join(type_save_path, "checkpoint.json")
temp_save_path = os.path.join(type_save_path, "temp_activations")
os.makedirs(temp_save_path, exist_ok=True)

# 加载模型
tokenizer = AutoTokenizer.from_pretrained(LLM_datapath, trust_remote_code=True)
model = AutoModel.from_pretrained(
    LLM_datapath,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 添加特殊标记
if tokenizer.bos_token is None:
    tokenizer.add_special_tokens({'bos_token': '<sop>'})
    model.resize_token_embeddings(len(tokenizer))

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

model = model.eval()

# 读取数据
prompt_df = pd.read_csv(prompt_data_path, header=0)
street_ids = list(prompt_df['Street_ID'])
dataset_strings = list(prompt_df['Prompt'])
print(f"Total prompts: {len(prompt_df)}")

def encode_single_prompt(text):
    """编码单个prompt"""
    encoded = tokenizer.encode(text, add_special_tokens=False, return_tensors='pt')
    # 添加BOS token
    bos_token_tensor = torch.tensor([[tokenizer.bos_token_id]], dtype=torch.long)
    token_ids = torch.cat([bos_token_tensor, encoded], dim=1)
    return token_ids

def extract_prompt_representation(activation, method='last_token'):
    """提取整个prompt的表示"""
    if method == 'last_token':
        return activation[0, -1, :]  # 最后一个token
    elif method == 'mean_pooling':
        return activation[0].mean(dim=0)  # 所有token的平均
    elif method == 'max_pooling':
        return activation[0].max(dim=0)[0]  # 所有token的最大值
    else:
        raise ValueError(f"Unknown method: {method}")

def load_checkpoint():
    """加载检查点"""
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as f:
            return json.load(f)
    return {"processed_indices": [], "last_processed": -1}

def save_checkpoint(processed_indices, last_processed):
    """保存检查点"""
    checkpoint = {
        "processed_indices": processed_indices,
        "last_processed": last_processed
    }
    with open(checkpoint_path, 'w') as f:
        json.dump(checkpoint, f)

def save_intermediate_batch(activations_batch, street_ids_batch, batch_idx):
    """保存中间批次"""
    torch.save({
        'activations': torch.stack(activations_batch, dim=0),
        'street_ids': street_ids_batch
    }, os.path.join(temp_save_path, f'batch_{batch_idx}.pt'))

# 加载检查点
checkpoint = load_checkpoint()
processed_indices = set(checkpoint["processed_indices"])
start_idx = checkpoint["last_processed"] + 1

print(f"Resuming from index {start_idx}, already processed: {len(processed_indices)} samples")

# 处理激活值 - 增加中间保存
with torch.no_grad():
    batch_size = 1000  # 每1000个样本保存一次
    current_batch_activations = []
    current_batch_street_ids = []
    batch_idx = 0

    for idx in tqdm(range(start_idx, len(street_ids)), desc="Processing prompts"):
        # 跳过已处理的样本
        if idx in processed_indices:
            continue

        street_id = street_ids[idx]
        prompt_text = dataset_strings[idx]

        if len(current_batch_activations) > 0:
            torch.cuda.empty_cache()

        # 编码prompt
        token_ids = encode_single_prompt(prompt_text).to(model.device)

        print(f"Processing sample {idx+1}/{len(street_ids)}, sequence length: {token_ids.shape[1]}")

        # 前向传播
        out = model(token_ids,
                   output_hidden_states=True,
                   output_attentions=False,
                   return_dict=True,
                   use_cache=False)

        # 只提取最后一层的激活值
        last_layer_activation = out.hidden_states[-1]  # 最后一层
        last_layer_activation = last_layer_activation.cpu().to(torch.float16)

        # 提取整个prompt的表示（使用最后一个token）
        prompt_repr = extract_prompt_representation(last_layer_activation, method='last_token')

        # 添加到当前批次
        current_batch_activations.append(prompt_repr)
        current_batch_street_ids.append(street_id)
        processed_indices.add(idx)

        # 清理
        del out, token_ids, last_layer_activation
        torch.cuda.empty_cache()

        # 达到批次大小或最后一个样本时保存
        if len(current_batch_activations) >= batch_size or idx == len(street_ids) - 1:
            save_intermediate_batch(current_batch_activations, current_batch_street_ids, batch_idx)
            print(f"Saved batch {batch_idx} with {len(current_batch_activations)} samples")

            # 保存检查点
            save_checkpoint(list(processed_indices), idx)

            # 清空当前批次
            current_batch_activations = []
            current_batch_street_ids = []
            batch_idx += 1

# 合并所有批次文件
print("Merging all batches...")
all_activations = []
all_street_ids = []

for i in range(batch_idx):
    batch_file = os.path.join(temp_save_path, f'batch_{i}.pt')
    if os.path.exists(batch_file):
        batch_data = torch.load(batch_file)
        all_activations.append(batch_data['activations'])
        all_street_ids.extend(batch_data['street_ids'])

if all_activations:
    final_activations = torch.cat(all_activations, dim=0)

    # 保存最终结果
    torch.save({
        'activations': final_activations,
        'street_ids': all_street_ids
    }, os.path.join(type_save_path, 'all_activations.pt'))

    print(f"Activations saved to {type_save_path}")
    print(f"Final activation shape: {final_activations.shape}")

    # 清理临时文件
    import shutil
    shutil.rmtree(temp_save_path)
    os.remove(checkpoint_path)
    print("Temporary files cleaned up")
else:
    print("No activations to save")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Total prompts: 5458
Resuming from index 0, already processed: 0 samples


Processing prompts:   0%|          | 0/5458 [00:00<?, ?it/s]

Processing sample 1/5458, sequence length: 1394


Processing prompts:   0%|          | 1/5458 [00:01<1:32:37,  1.02s/it]

...

Processing sample 5453/5458, sequence length: 1430
Processing sample 5454/5458, sequence length: 934


Processing prompts: 100%|█████████▉| 5456/5458 [19:31<00:00,  9.94it/s]

Processing sample 5455/5458, sequence length: 584
Processing sample 5456/5458, sequence length: 562
Processing sample 5457/5458, sequence length: 1782
Processing sample 5458/5458, sequence length: 1795
Saved batch 5 with 458 samples


Processing prompts: 100%|██████████| 5458/5458 [19:32<00:00,  4.66it/s]

Merging all batches...
Activations saved to /content/drive/MyDrive/Qwen3-8B_result
Final activation shape: torch.Size([5458, 4096])
Temporary files cleaned up
